In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib
matplotlib.rcParams['timezone'] = 'US/Central'

import pickle
import pandas as pd
import numpy as np
import datetime as dt
import itertools as it
import math
import pytz
import sys
import warnings
import os
from IPython.display import display, HTML


import util

from decimal import Decimal
from poloniex import poloniex



cst = pytz.timezone('America/Chicago')

secret = 'd2604c1de9dc757984746e78551936ce53d4f9dd72441e05a5cf61606bd469b6ea41bd1f95a82f08083f220a7e0f89bf018d3f34fa6d9b8e8a7815dace02bd1e'
key    = 'IZQJ94DS-SFJ2D0FW-LK2FUCBX-29FZMFWU'

polo = poloniex(key, secret)

def load_ticker(ticker, start, end, period):
    new_data = pd.DataFrame.from_dict(polo.returnChartData(ticker, start, end, period))
    data = new_data[['weightedAverage']]
    data.index=[dt.datetime.utcfromtimestamp(d) for d in new_data['date']]
    return data


# Create realized volatility index

tickers= ['USDT_BTC','USDT_ETH','USDT_BCH','USDT_XRP']  

period = 86400 # time interval at which price is taken (in seconds) i..e 300 = price snapshots every 5 minutes

timeframe= 30 # Number of days (i.e. 7 = Weekly Index)


In [2]:
end  =   dt.datetime.utcnow().date()

start = dt.datetime.utcnow().date()-dt.timedelta(days=timeframe*3)

tickers_data=pd.DataFrame()
for ticker in tickers:
    daily_data=pd.DataFrame()
    days = pd.date_range(start, end, freq='D')
    for d1 in days:
        f_name = '{0} {1}.csv'.format(ticker, str(d1.date()))
        if d1==days[-1]:
            print('\rdownloading: '+f_name, end='', flush=True)
            data = load_ticker(ticker, d1, d1+dt.timedelta(days=1)-dt.timedelta(seconds=period), period)
            data.columns = [ticker]
            data.to_csv('daily_data/'+ f_name, index_label='time')
    #     elif f_name in os.listdir('daily_data/'):
    #         print('\rloading: '+f_name, end='', flush=True)
    #         data = pd.read_csv('daily_data/'+f_name, parse_dates=True, index_col=0)
        else:
            print('\rdownloading: '+f_name, end='', flush=True)
            data = load_ticker(ticker, d1, d1+dt.timedelta(days=1)-dt.timedelta(seconds=period), period)
            data.columns = [ticker]
            data.to_csv('daily_data/'+ f_name, index_label='time')
        daily_data=pd.concat([daily_data, data])
    tickers_data=pd.concat([tickers_data,daily_data], axis=1)
    

downloading: USDT_XRP 2018-08-19.csv

In [3]:
import numpy as np
vol_data=pd.DataFrame()
for ticker in tickers:
    tickers_data[str(ticker[-3:])+' pct change']= tickers_data[ticker].pct_change()
    tickers_data[str(ticker[-3:])+' log_returns']=np.log(tickers_data[ticker]) - np.log(tickers_data[ticker].shift(1))


    tickers_data[str(ticker[-3:]+ ' Volatility')]=''

    for i,k in zip(range(len(tickers_data)),tickers_data.index):
        tickers_data.loc[k,(str(ticker[-3:]+ ' Volatility'))]= tickers_data.iloc[i-(timeframe-1):i+1][str(ticker[-3:]) +' log_returns'].std()*math.sqrt(365)

    volatility=tickers_data[[str(ticker[-3:]+ ' Volatility')]]
    
    vol_data=pd.concat([vol_data,volatility],axis=1)

    
#     display(HTML(daily_data[-20:].to_html()))


#     util.quick_plot(daily_data['Volatility'][:])

In [4]:
import requests
grab_url='https://api.coinmarketcap.com/v2/ticker/?limit=4' 
req = requests.get(grab_url)
output = req.json()


top_5 = [item for item in output['data'].keys()]

top_tickers = [output['data'][name]['name'] for name in top_5]


market_cap={key:output['data'][key]['quotes']['USD']['market_cap'] for key in top_5} 

total_top_5=[item for item in market_cap.values()]

data= pd.DataFrame(total_top_5,columns=["Market Cap"],index=['1','2','3','4'])
data['Total MC']= data['Market Cap'].sum()

for i in data.index:
    data['percentage']= data['Market Cap']/data['Total MC']


# for i,k in zip(range(len(data)),data.index):
#     daily_data.loc[k,('Volatility')]= daily_data.iloc[i-(timeframe-1):i+1].log_returns.std()*math.sqrt(365)
    
    

data.index=['BTC','ETH','XRP','BCH']


value=0
for n in data.index:
    vol_data[str(n)+'Standard']=vol_data[str(n)+' Volatility']*data.loc[n]['percentage']
    value+=vol_data[str(n)+'Standard']
vol_data['Total']=value

In [7]:
vol_data[-15:]

,BTC Volatility,ETH Volatility,BCH Volatility,XRP Volatility,BTCStandard,ETHStandard,XRPStandard,BCHStandard,Total
2018-08-05,0.54225,0.512562,0.647934,0.480351,0.3659,0.0944409,0.0390778,0.0386254,0.538044
2018-08-06,0.541421,0.508573,0.642352,0.485863,0.365341,0.0937058,0.0395263,0.0382926,0.536866
2018-08-07,0.538807,0.508798,0.619689,0.511198,0.363577,0.0937473,0.0415873,0.0369417,0.535853
2018-08-08,0.602236,0.555676,0.739411,0.73026,0.406378,0.102385,0.0594086,0.0440787,0.61225
2018-08-09,0.586871,0.501249,0.722061,0.72346,0.396009,0.0923565,0.0588554,0.0430444,0.590266
2018-08-10,0.590379,0.532468,0.728165,0.744366,0.398377,0.0981086,0.0605562,0.0434083,0.60045
2018-08-11,0.586134,0.57826,0.732454,0.768576,0.395512,0.106546,0.0625258,0.0436639,0.608248
2018-08-12,0.590809,0.578492,0.727481,0.765842,0.398667,0.106589,0.0623034,0.0433675,0.610927
2018-08-13,0.590803,0.626392,0.731404,0.767004,0.398663,0.115414,0.0623979,0.0436014,0.620076
2018-08-14,0.610703,0.705164,0.824471,0.841184,0.412091,0.129928,0.0684326,0.0491494,0.659602
